# David Fleming
# ASTR 597b
# Prof. Juric
# March 12th, 2016

In [1]:
# Imports
import numpy as np
from astropy.table import Table
from lsd import DB
import os

### While working on magneto, limit LSD's core usage and let it know where important directories are.

In [2]:
os.environ['NWORKERS'] = '8'
os.environ['LSD_DB'] = "/astro/users/dflemin3/tempdir"
os.environ['LSD_CACHEDIR'] = "/astro/users/dflemin3/tempdir"
os.environ['LSD_TEMPDIR'] = "/astro/users/dflemin3/tempdir"
datadir = "/astro/users/dflemin3/Desktop/"

### Load in representative light curve and pixel file data.  For this demonstration, we will load in data for two objects using the two data outputs provided by the K2 mission, the processed light curve and the raw pixel file data.  For K2, the processed light curve is not really useful as most groups write pipelines for raw pixel file data, but it will still prove insightful to import both into LSD.

In [3]:
# Inspect the FITS files
k2lc_data = Table.read(datadir + "ktwo211402082-c05_llc.fits")
k2tar_data = Table.read(datadir + "ktwo200000811-c00_lpd-targ.fits")

### Print out the column names and their datatypes as these must be known to construct the .yaml files.

In [4]:
# Lightcurve
# Print out the column names and types
for column in k2lc_data.dtype.names:
    print "(%s, %s)" % (column.lower(), k2lc_data[column].dtype)

(time, >f8)
(timecorr, >f4)
(cadenceno, >i4)
(sap_flux, >f4)
(sap_flux_err, >f4)
(sap_bkg, >f4)
(sap_bkg_err, >f4)
(pdcsap_flux, >f4)
(pdcsap_flux_err, >f4)
(sap_quality, >i4)
(psf_centr1, >f8)
(psf_centr1_err, >f4)
(psf_centr2, >f8)
(psf_centr2_err, >f4)
(mom_centr1, >f8)
(mom_centr1_err, >f4)
(mom_centr2, >f8)
(mom_centr2_err, >f4)
(pos_corr1, >f4)
(pos_corr2, >f4)


In [5]:
# Target pixel file
# Print out the column names and types
for column in k2tar_data.dtype.names:
    print "(%s, %s)" % (column.lower(), k2tar_data[column].dtype)

(time, >f8)
(timecorr, >f4)
(cadenceno, >i4)
(raw_cnts, >i4)
(flux, >f4)
(flux_err, >f4)
(flux_bkg, >f4)
(flux_bkg_err, >f4)
(cosmic_rays, >f4)
(quality, >i4)
(pos_corr1, >f4)
(pos_corr2, >f4)


### After writing .yaml files, display them to ensure that they are in the proper format.

In [6]:
! cat k2_lc.yaml

filters: {complevel: 5, complib: blosc}
schema:
  common:
    primary_key: obj_id
    spatial_keys: [obj_id, cadenceno]
    columns:
    - [obj_id, u8]
    - [time, f8]
    - [timecorr, f4]
    - [cadenceno, i4]
  photometry:
    columns:
    - [sap_flux, f4]
    - [sap_flux_err, f4]
    - [sap_bkg, f4]
    - [sap_bkg_err, f4]
    - [pdcsap_flux, f4]
    - [pdcsap_flux_err, f4]
    - [sap_quality, i4]
    - [psf_centr1, f8]
    - [psf_centr1_err, f4]
    - [psf_centr2, f8]
    - [psf_centr2_err, f4]
  telemetry:
    columns:
    - [mom_centr1, f8]
    - [mom_centr1_err, f4]
    - [mom_centr2, f8]
    - [mom_centr2_err, f4]
    - [pos_corr1, f4]
    - [pos_corr2, f4]

In [7]:
! cat k2_tar.yaml

filters: {complevel: 5, complib: blosc}
schema:
  common:
    primary_key: obj_id
    spatial_keys: [obj_id, cadenceno]
    columns:
    - [obj_id, u8]
    - [time, f8]
    - [timecorr, f4]
    - [cadenceno, i4]
    - [quality, i4]
    - [pos_corr1, f4]
    - [pos_corr2, f4]
  photometry:
    columns:
    - [raw_cnts, '(50,50)i4']
    - [flux, '(50,50)f4']
    - [flux_err, '(50,50)f4']
    - [flux_bkg, '(50,50)f4']
    - [flux_bkg_err, '(50,50)f4']
    - [cosmic_rays, '(50,50)f4']

### Create the two tables, dropping them if they already exist.

In [8]:
# Create the table
! lsd-admin create table --drop-existing --schema=k2_lc.yaml k2_lightcurve

Table 'k2_lightcurve' dropped.

-------- committing 20160313015157.329791 [k2_lightcurve] ---------
[k2_lightcurve] Updating tablet catalog: [256 el.]::::::::::::::::::::>  0.27 sec
[k2_lightcurve] Updating neighbors: Already up to date.
[k2_lightcurve] Updating tablet catalog: [256 el.]::::::::::::::::::::>  0.25 sec
[k2_lightcurve] Updating stats: [0 el.]>  0.00 sec
[k2_lightcurve] Marking tablets read-only...
----------- success 20160313015157.329791 [k2_lightcurve] ---------

Table 'k2_lightcurve' created.


In [9]:
# Create the table
! lsd-admin create table --drop-existing --schema=k2_tar.yaml k2_targ

Table 'k2_targ' dropped.

-------- committing 20160313015202.902391 [k2_targ] ---------
[k2_targ] Updating tablet catalog: [256 el.]::::::::::::::::::::>  0.23 sec
[k2_targ] Updating neighbors: Already up to date.
[k2_targ] Updating tablet catalog: [256 el.]::::::::::::::::::::>  0.24 sec
[k2_targ] Updating stats: [0 el.]>  0.00 sec
[k2_targ] Marking tablets read-only...
----------- success 20160313015202.902391 [k2_targ] ---------

Table 'k2_targ' created.


### Import the data!

In [10]:
! lsd-import fits k2_lightcurve /astro/users/dflemin3/Desktop/ktwo211402082-c05_llc.fits

Importing from 1 pieces:
  ===> Imported /astro/users/dflemin3/Desktop/ktwo211402082-c05_llc.fits               [1/1, 100.00%] +   3663/3663         3663 (0/0 min.)
done

-------- committing 20160313015206.377434 [k2_lightcurve] ---------
[k2_lightcurve] Updating tablet catalog: [256 el.]::::::::::::::::::::>  0.46 sec
[k2_lightcurve] Updating neighbors: [49 el.]::::::::::::::::::::[203 el.]++++++++++++++++++++>  2.65 sec
                Total 33544 cached objects in 203 cells
[k2_lightcurve] Updating tablet catalog: [256 el.]::::::::::::::::::::>  0.37 sec
[k2_lightcurve] Updating stats: [203 el.]::::::::::::::::::::>  0.06 sec [203 el.]::::::::::::::::::::>  0.26 sec
[k2_lightcurve] Marking tablets read-only...
----------- success 20160313015206.377434 [k2_lightcurve] ---------



In [11]:
! lsd-import fits k2_targ /astro/users/dflemin3/Desktop/ktwo200000811-c00_lpd-targ.fits

Importing from 1 pieces:
  ===> Imported /astro/users/dflemin3/Desktop/ktwo200000811-c00_lpd-targ.fits          [1/1, 100.00%] +   3753/3753         3753 (0/0 min.)
done

-------- committing 20160313015216.842732 [k2_targ] ---------
[k2_targ] Updating tablet catalog: [256 el.]::::::::::::::::::::>  0.28 sec
[k2_targ] Updating neighbors: [49 el.]::::::::::::::::::::[203 el.]++++++++++++++++++++>  19.69 sec
          Total 34469 cached objects in 203 cells
[k2_targ] Updating tablet catalog: [256 el.]::::::::::::::::::::>  0.39 sec
[k2_targ] Updating stats: [203 el.]::::::::::::::::::::>  0.05 sec [203 el.]::::::::::::::::::::>  0.29 sec
[k2_targ] Marking tablets read-only...
----------- success 20160313015216.842732 [k2_targ] ---------

